<a href="https://colab.research.google.com/github/anuva04/ML_Beginners/blob/main/artificial_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Network

## The Neuron
- the basic building block of neural networks
- *neuron and node are used interchangeably*
- it gets some input signals/values
- it gives an output signal/value
- the input variables are independent, but for only one specific observation. However, they need to be standardised or normalised
- output value can be binary, continuous or categorical
- the input values are multiplied with some weights, and these are fundamental to the learning process of the neurons
- all the input values are multiplied with weights are summed up and fed into an activation function
- the activation function decides whether to pass on the signal or not

## Activation function
- each neuron has an activation function
- The common activation functions are:
- threshold function: if signal is less than threshold, pass on 0, else pass on 1
- sigmoid function: 1/(1+e^-x)
- rectifier function: max(0,x) -> for values less than 0, it outputs 0, and otherwise it outputs the actual value
- hyperbolic tangent function: (1-e^(-2x))/(1+e^(-2x))

## How do NNs learn?
- some input values are fed into the model
- the model assigns some weights to them
- it calculates the output based on the weights and activation function
- it compares this output with the actual output 
- it calulates the error using a cost function
- the error value is fed back into the model
- based on this value, the assigned weights are updated to minimise the error
- this process is called back propagation

## Gradient Descent
- the cost function is: **C = 1/2(y-y')^2**
- it is a quadratic function with a minimum value, which are trying to achieve
- in the cost function curve, we start at a point and determine the slope
- if slope is negative, means we need to go right, else we need to go left
- it is called gradient descent because we are descending towards the minimum of the cost function
- for this method to work, the cost function should be convex, that is it should have one global minimum
- if there are multiple local minima, the model may find a non-optimal minimum which is not the global minimum
- to solve this issue, stochastic gradient descent is used
- in batch gradient descent, all the rows of data is fed at once, and then weights are adjusted
- however, in stochastic gradient descent, at a time only one row is processed, the weights are adjusted and then the next row is processed
- stochastic gradient descent is much more likely to find the global minimum
- batch gradient descent is a deterministic algorithm, whereas stochastic gradient descent is stochastic or random
- mini batch gradient descent is a hybrid of the above two, where mini batches of a section of rows are processed at a time
<hr>
- **backpropagation** is the process of sending back cost function information to the input layer in order to adjust weights
- **learning rate** decides how much the weights are updated after each backpropagation
- when the whole training set is processed by the ANN model, it is called one **epoch**




### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.6.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [4]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [5]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [7]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [9]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling
- feature scaling is a must for deep learning
- apply it to all the features

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN
- initalise ANN as a sequence of layers

In [12]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer
- units: number of neurons in the layer
- the number of units have to selected by experimentation
- here, the activation function is a rectifier function

In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [14]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer
- since the output column of this dataset is a binary variable, we need only one variable to encode it, which will take values 0 or 1
- hence, we need only one neuron in the output layer
- moreover, since the output is binary, a sigmoid function will be a good fit for activation function

In [15]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN
- optimiser is used to perform stochastic gradient descent
- loss is used to define the loss function which is the cost function
- for computing a binary outcome, loss function is binary_crossentropy
- for computing a outcome with more than 2 variables, loss function to be used is catergorical_crossentropy
- the accuracy is calculated based on the metrics chosen

In [16]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [17]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 789us/step - loss: 0.6281 - accuracy: 0.6862
Epoch 2/100
250/250 [==============================] - 0s 886us/step - loss: 0.4734 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 857us/step - loss: 0.4459 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 843us/step - loss: 0.4339 - accuracy: 0.7964
Epoch 5/100
250/250 [==============================] - 0s 884us/step - loss: 0.4242 - accuracy: 0.7995
Epoch 6/100
250/250 [==============================] - 0s 806us/step - loss: 0.4130 - accuracy: 0.8062
Epoch 7/100
250/250 [==============================] - 0s 852us/step - loss: 0.3998 - accuracy: 0.8148
Epoch 8/100
250/250 [==============================] - 0s 792us/step - loss: 0.3894 - accuracy: 0.8219
Epoch 9/100
250/250 [==============================] - 0s 805us/step - loss: 0.3820 - accuracy: 0.8298
Epoch 10/100
250/250 [==============================] - 0s 838us/step - l

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [18]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [19]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1502   93]
 [ 191  214]]


0.858